In [5]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, RepeatVector
from tensorflow.keras.callbacks import ModelCheckpoint

# Define parameters
timesteps = 10
input_dim = 1
latent_dim = 2

# Generate synthetic data
data = np.random.rand(1000, timesteps, input_dim)

# Define the autoencoder model
inputs = Input(shape=(timesteps, input_dim))
encoded = LSTM(4)(inputs)
encoded = RepeatVector(timesteps)(encoded)
decoded = LSTM(4, return_sequences=True)(encoded)
decoded = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(input_dim))(decoded)
autoencoder = Model(inputs, decoded)
autoencoder.compile(optimizer='adam', loss='mse')
print("model summary")
autoencoder.summary()

# Train the autoencoder
autoencoder.fit(data, data, epochs=5, batch_size=32, validation_split=0.2)

# Define the encoder model
encoder = Model(inputs, encoded)

# Define the decoder model
encoded_input = Input(shape=(timesteps, 4))
decoder_layer = autoencoder.layers[-2](encoded_input)
decoder_layer = autoencoder.layers[-1](decoder_layer)
decoder = Model(encoded_input, decoder_layer)

def encode_and_decode(sequence):
    # Preprocess the input sequence
    sequence = np.array(sequence).reshape(1, timesteps, input_dim)

    # Encode the sequence
    encoded_sequence = encoder.predict(sequence)

    # Decode the sequence
    decoded_sequence = decoder.predict(encoded_sequence)

    return encoded_sequence, decoded_sequence

# Get user input
def get_user_sequence():
    try:
        user_input = input(f"Enter a sequence of {timesteps} numbers (comma-separated): ")
        sequence = [float(x) for x in user_input.split(',')]
        if len(sequence) != timesteps:
            raise ValueError(f"Please enter exactly {timesteps} numbers.")
        return sequence
    except ValueError as e:
        print(e)
        return None

# Main function to handle user input
def main():
    sequence = get_user_sequence()
    if sequence is not None:
        encoded_sequence, decoded_sequence = encode_and_decode(sequence)
        print("Encoded Sequence:")
        print(encoded_sequence)
        print("Decoded Sequence:")
        print(decoded_sequence.reshape(timesteps, input_dim))

if __name__ == "__main__":
    main()


model summary


Model: "functional_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_7 (InputLayer)           │ (None, 10, 1)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_8 (LSTM)                        │ (None, 4)                   │              96 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ repeat_vector_4 (RepeatVector)       │ (None, 10, 4)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_9 (LSTM)                        │ (None, 10, 4)               │             144 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_3 (TimeDistributed) │ (None, 10, 1)               │               5 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 245 (980.00 B)

 Trainable params: 245 (980.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 30ms/step - loss: 0.2471 - val_loss: 0.1664
Epoch 2/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.1503 - val_loss: 0.0970
Epoch 3/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0983 - val_loss: 0.0920
Epoch 4/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0944 - val_loss: 0.0900
Epoch 5/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0933 - val_loss: 0.0888
Enter a sequence of 10 numbers (comma-separated): 1,2,3,4,5,6,7,8,9,12


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 271ms/step
Encoded Sequence:
[[[0.61541605 0.30174804 0.07126106 0.8636233 ]
  [0.61541605 0.30174804 0.07126106 0.8636233 ]
  [0.61541605 0.30174804 0.07126106 0.8636233 ]
  [0.61541605 0.30174804 0.07126106 0.8636233 ]
  [0.61541605 0.30174804 0.07126106 0.8636233 ]
  [0.61541605 0.30174804 0.07126106 0.8636233 ]
  [0.61541605 0.30174804 0.07126106 0.8636233 ]
  [0.61541605 0.30174804 0.07126106 0.8636233 ]
  [0.61541605 0.30174804 0.07126106 0.8636233 ]
  [0.61541605 0.30174804 0.07126106 0.8636233 ]]]
Decoded Sequence:
[[0.4143814 ]
 [0.6511851 ]
 [0.80063623]
 [0.8945565 ]
 [0.9544173 ]
 [0.9933041 ]
 [1.0190678 ]
 [1.0364654 ]
 [1.0484288 ]
 [1.0567989 ]]
